In [ ]:
    def equation_trajectoire(self, x : float | np.ndarray, Bz : float) -> float | np.ndarray :
        """
        La position y de la particule en x. Retourne NaN si x n'est pas physiquement atteignable.

        Parameters
        ----------
        x : float or np.ndarray
            Position(s) en x de la particule (en m)
        Bz : float
            Valeur du champ magnétique d'axe z (en T)

        Returns
        -------
        float or np.ndarray
            Position(s) en y de la particule (en m), ou NaN si x est hors de portée.
        """
        # --- MODIFICATION OBLIGATOIRE : Gestion domaine arccos ---
        prefix = self.mq / Bz
        rayon = abs(self.vo * prefix) # Rayon de la trajectoire circulaire
        max_x_physique = 2 * rayon # La particule ne peut pas aller au delà de x=2R

        # Crée un masque pour les x valides (0 <= x <= max_x_physique)
        # Important : Gère les cas où x est un scalaire ou un ndarray
        if isinstance(x, np.ndarray):
            y = np.full_like(x, np.nan) # Initialiser le résultat avec NaN
            mask = (x >= 0) & (x <= max_x_physique)
            valid_x = x[mask]

            if valid_x.size > 0: # Calculer seulement pour les x valides
                 # Calcul de l'argument de arccos
                 arg = 1 - valid_x / (self.vo * prefix) # Attention R = vo*prefix si qB>0
                                                        # Si qB<0, R = -vo*prefix
                                                        # La formule originale utilise vo*prefix, gardons cela
                 # Clip l'argument pour éviter les erreurs de précision flottante aux bords
                 clipped_arg = np.clip(arg, -1.0, 1.0)
                 # Calculer y pour les x valides
                 y[mask] = self.vo * prefix * np.sin(np.arccos(clipped_arg))
            return y
        else: # Traitement pour x scalaire
            if 0 <= x <= max_x_physique:
                arg = 1 - x / (self.vo * prefix)
                clipped_arg = np.clip(arg, -1.0, 1.0)
                return self.vo * prefix * np.sin(np.arccos(clipped_arg))
            else:
                return np.nan # Retourne NaN si x est hors de la plage physique
        # --- FIN MODIFICATION ---

In [ ]:

# Objectif 1

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from scipy.optimize import fsolve
import scipy.constants as constants


# Objet d'une particule décrite avec son rapport masse/charge et sa vitesse initiale
class particule :
    def __init__(self, masse_charge : tuple[int, int], v_initiale : float) -> None :
        """
        Objet particule traversant un champ magnétique d'axe z

        Parameters
        ----------
        masse_charge : tuple of int
            Masse (en unités atomiques), Charge (en unités de charge élémentaire, e) de la particule.
        v_initiale : float
            Vitesse initiale en y de la particule (en m/s).
        """
        # --- MODIFICATION OBLIGATOIRE ---
        mass_u = masse_charge[0]
        charge_e = masse_charge[1]
        if charge_e == 0:
             raise ValueError("La charge de la particule ne peut pas être nulle.")
        # Utilisation de constants.e (charge élémentaire en C) au lieu de constants.eV
        self.mq = (mass_u * constants.u) / (charge_e * constants.e) # kg/C
        # --- FIN MODIFICATION ---

        self.vo = v_initiale
        self.m = mass_u
        self.c = charge_e # Garde la charge en unités 'e' pour l'affichage


    # Niveau 5 : L'equation de la trajectoire d'une particule en fonction de son rapport masse/charge et sa vitesse initiale
    def equation_trajectoire(self, x : float, Bz : float) -> float :
        """
        La position y de la particule en x

        Parameters
        ----------
        x : float
            Position en x de la particule (en m)
        Bz : float
            Valeur du champ magnétique d'axe z (en T)

        Returns
        -------
        float
            Position en y de la particule (en m)
        """
        prefix = self.mq / Bz # C'est R = rayon de Larmor * signe(q*Bz)
        arg_arccos = 1 - x / (self.vo * prefix)

        # --- Gestion des erreurs arccos ---
        # Clipper l'argument pour éviter les erreurs dues aux imprécisions numériques
        arg_arccos = np.clip(arg_arccos, -1.0, 1.0)
        # --- Fin Gestion ---

        # L'angle phi de rotation est arccos(1 - x/R)
        # y = R * sin(phi)
        return self.vo * prefix * np.sin(np.arccos(arg_arccos))


    # Niveau 4 : Renvoie un tuple de la trajectoire de la particule (liste des abscisses, liste des ordonnées)
    def trajectoire(self, Bz : float, x_min : float, x_max : float, n_points : int = 10000) -> tuple[np.ndarray, np.ndarray] :
        """
        Calcule la trajectoire entre un x minimum et un x maximum

        Parameters
        ----------
        Bz : float
            Valeur du champ magnétique d'axe z (en T)
        x_min : float
            Position en x minimale (en m)
        x_max : float
            Position en x maximale (en m)
        n_points : int
            Nombre de points où la position sera calculée entre x_min et x_max

        Returns
        -------
        tuple of (numpy.ndarray, numpy.ndarray)
            - Positions en x
            - Positions en y

        """
        x = np.linspace(x_min, x_max, n_points)
        y = self.equation_trajectoire(x, Bz)
        # Filtrer les NaN qui pourraient apparaître si Bz=0 ou si x > diamètre
        mask = ~np.isnan(y)
        return x[mask], y[mask]


    # Niveau 3 : Trace la trajectoire de la particule dans le champ Bz avec matplotlib en 2d
    def tracer_trajectoire(self, ax, Bz : float, x_min : float, x_max : float, n_points : int = 10000) -> None :
        """
        Trace la trajectoire entre x_min et x_max sur ax

        Parameters
        ----------
        ax : matplotlib.axes.Axes
            L'axe matplotlib sur lequel on veut tracer la trajectoire
        Bz : float
            Valeur du champ magnétique d'axe z (en T)
        x_min : float
            Position en x minimale (en m)
        x_max : float
            Position en x maximale (en m)
        n_points : int
            Nombre de points où la position sera calculée entre x_min et x_max

        """
        x, y = self.trajectoire(Bz, x_min, x_max, n_points)
        if len(x) > 0:
            # --- MODIFICATION OBLIGATOIRE ---
            # Correction du label pour afficher 'e' au lieu de 'eV'
            ax.plot(x, y, label=f'{self.m}u, {self.c:+}e') # :+ pour afficher le signe de la charge
            # --- FIN MODIFICATION ---


    # Niveau 2.1 : Détermine la puissance du champ magnétique nécéssaire pour dévier une particule à un point précis
    def determiner_champ_magnetique(self, x_objective : float, y_objective : float, B0 : float = None) -> float :
        """
        Donne le champ magnétique pour dévier la particule en (x_objective, y_objective) depuis l'origine

        Parameters
        ----------
        x_objective : float
            Position en x voulue à l'état final
        y_objective : float
            Position en y voulue à l'état final
        B0 : float
            Valeur de départ de recherche du champ magnétique (pour la fonction fsolve de scipy)

        Returns
        -------
        float
            Champ magnétique (en T)
        """
        # Note: fsolve peut avoir des difficultés si l'équation a plusieurs solutions ou des discontinuités.
        # La valeur initiale B0 peut être importante.
        # Si mq est négatif, le sens de déviation change.
        if B0 is None :
            # Heuristique simple pour B0 : basé sur le rayon R approx y_objective (si x petit) ou x_objective/2 (si demi-cercle)
            # R = vo * mq / B => B = vo * mq / R
            # Utilisons une estimation basée sur R ~ sqrt(x^2+y^2) ? Non, plus complexe.
            # Tentons une valeur basée sur mq, mais en valeur absolue pour éviter B0 négatif
            B0 = abs(self.vo * self.mq / max(x_objective, y_objective, 1e-6)) # Estimation grossière
            if B0 == 0: B0 = 1e-3 # Éviter B0 nul

        # Définir l'équation à résoudre : y_calc(B) - y_objective = 0
        def equation_func(B):
            if abs(B) < 1e-15: return np.inf # Éviter division par zéro
            prefix = self.mq / B
            arg_arccos = 1 - x_objective / (self.vo * prefix)
            # Vérifier le domaine de arccos
            if not (-1.0 <= arg_arccos <= 1.0):
                # Retourner une grande valeur si hors domaine pour guider fsolve
                return 1e10 * np.sign(arg_arccos)
            y_calc = self.vo * prefix * np.sin(np.arccos(arg_arccos))
            return y_calc - y_objective

        # Utiliser fsolve pour trouver B
        B_solution, infodict, ier, mesg = fsolve(equation_func, B0, full_output=True)

        # Vérifier si fsolve a convergé
        if ier == 1:
            return B_solution[0]
        else:
            print(f"Avertissement: fsolve n'a pas convergé pour déterminer Bz ({mesg})")
            # Essayer une autre valeur initiale?
            # Tenter avec -B0 si la première tentative échoue?
            B0_alt = -B0
            B_solution_alt, infodict_alt, ier_alt, mesg_alt = fsolve(equation_func, B0_alt, full_output=True)
            if ier_alt == 1:
                 print(f"   -> Convergence réussie avec B0={B0_alt:.2e}")
                 return B_solution_alt[0]
            else:
                 print(f"   -> Échec aussi avec B0={B0_alt:.2e}")
                 return np.nan # Retourner NaN si aucune solution trouvée

# Niveau 2.2 : Tracer l'ensemble des trajectoires des particules d'un faisceau
def tracer_ensemble_trajectoires(masses_charges_particules : list[tuple[int, int]], vitesse_initiale : float, Bz : float, x_detecteur : float, create_plot : bool = True, ax = None) -> None:
    """
    Trace les trajectoires entre 0 et x_detecteur pour un ensemble de particules d'un faisceau

    Parameters
    ----------
    masses_charges_particules : list of tuple of int
        Masse (en unités atomiques), Charge (en eV)  pour toutes les particules
    vitesse_initiale : float
        Vitesse intiale en y commune à toutes les particules du faisceau
    Bz : float
            Valeur du champ magnétique d'axe z (en T)
    x_detecteur : float
        L'abscisse du détecteur (en m)
    """
    particules = [particule(masse_charge, vitesse_initiale) for masse_charge in masses_charges_particules]    # Liste d'objets particule représentant toutes les particules
    if ax == None or create_plot == True :
        fig, ax = plt.subplots()
    
    all_y_contact = []
    for particule_locale in particules :
        particule_locale.tracer_trajectoire(ax, Bz, 0, x_detecteur)
        all_y_contact.append(particule_locale.equation_trajectoire(x_detecteur, Bz))

    ax.plot([x_detecteur, x_detecteur], [min(all_y_contact) * 0.8, max(all_y_contact) * 1.1], c='black', linewidth=5, label='Détecteur')
    ax.set_xlabel('Position x (en m)')
    ax.set_ylabel('Position y (en m)')
    ax.legend()
    if create_plot :
        plt.show()


def tracer_trajectoires_dynamiquement(masses_charges_particules : list[tuple[int, int]], vi_min : float, vi_max : float, Bz_min : float, Bz_max : float, x_detecteur : float, create_plot : bool = True, fig = None, ax = None) -> None:
    """
    Trace les trajectoires entre 0 et x_detecteur pour un ensemble de particules d'un faisceau de manière dynamique

    Parameters
    ----------
    masses_charges_particules : list of tuple of int
        Masse (en unités atomiques), Charge (en eV)  pour toutes les particules
    vi_min : float
        Vitesse intiale en y minimale (en m/s)
    vi_max : float
        Vitesse intiale en y maximale (en m/s)
    Bz_min : float
        Valeur minimale du champ magnétique d'axe z (en T)
    Bz_max : float
        Valeur maximale du champ magnétique d'axe z (en T)
    x_detecteur : float
        L'abscisse du détecteur (en m)
    """
    particules = [particule(masse_charge, 0.5 * (vi_min + vi_max)) for masse_charge in masses_charges_particules]
    if create_plot or ax == None : 
        fig, ax = plt.subplots()
        plt.subplots_adjust(left=0.1, bottom=0.25)
    
    all_y_contact = []
    all_lines = []
    Bz0 = 0.5 * (Bz_min + Bz_max)
    for particule_locale in particules :
        all_lines.append(particule_locale.tracer_trajectoire(ax, Bz0, 0, x_detecteur))
        all_y_contact.append(particule_locale.equation_trajectoire(x_detecteur, Bz0))

    detecteur = ax.plot([x_detecteur, x_detecteur], [min(all_y_contact) * 0.8, max(all_y_contact) * 1.1], c='black', linewidth=5, label='Détecteur')

    ax.legend()

    ax_a = plt.axes([0.1, 0.15, 0.8, 0.03])
    ax_b = plt.axes([0.1, 0.1, 0.8, 0.03])
    slider_a = Slider(ax_a, 'V0 (m/s)', vi_min, vi_max, valinit= 0.5 * (vi_min + vi_max))
    slider_b = Slider(ax_b, 'Bz (T)', Bz_min, Bz_max, valinit= Bz0)

    def update(val) :
        v0 = slider_a.val
        Bz = slider_b.val
        particules = [particule(masse_charge, v0) for masse_charge in masses_charges_particules]
        for i in range(len(all_lines)) :
            all_lines[i][0].set_ydata(particules[i].trajectoire(Bz, 0, x_detecteur)[1])
        fig.canvas.draw_idle()
    
    slider_a.on_changed(update)
    slider_b.on_changed(update)
    return ax
    plt.show()



'''
Test de la fonction tracer_ensemble_trajectoires (valeurs non représentatives)

On trace les trajectoires de particules avec des rapports m/q différents dans un champ magnétique donné
'''
if __name__ == '__main__' :
    rapports_masse_charge = [(1, 1), (2, 1), (3, 1)]
    vitesse_initiale = 1e7
    Bz = 1
    x_detecteur = 1e-4
    
    tracer_ensemble_trajectoires(rapports_masse_charge, vitesse_initiale, Bz, x_detecteur)


'''
Test de la fonction déterminer_champ_magnétique (valeurs non représentatives)

On cherche le champ magnétique pour dévier la trajectoire en x_max, x_max
Puis on trace la trajectoire jusqu'en x_max
On remarque que la particule finit effectivement à la position prévue
'''
# if __name__ == '__main__' :
#     rapports_masse_charge, vi = [(1, 1)], 1e7
#     p = particule(rapports_masse_charge[0], vi)
#     x_max = 4.95e-2
#     Bz = p.determiner_champ_magnetique(x_max, x_max)
#     print(Bz)
#     tracer_ensemble_trajectoires(rapports_masse_charge, vi, Bz, x_max)
    

"""
Test de la fonction tracer_trajectoires_dynamiquement (valeurs non représentatives)
"""
# if __name__ == '__main__' :
#     rapports_masse_charge = [(1, 1), (2, 1), (3, 1)]
#     vi_min, vi_max = 1e7, 1e8
#     Bz_min, Bz_max = 1, 5
#     x_detecteur = 4.95e-2
    
#     tracer_trajectoires_dynamiquement(rapports_masse_charge, vi_min, vi_max, Bz_min, Bz_max, x_detecteur)

In [ ]:
import sys
import tkinter as tk
from tkinter import ttk, messagebox, font
import numpy as np
import scipy.constants as constants
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk

# Correction des imports pour correspondre aux noms de fichiers fournis
sys.path.append("./SIMS/Partie Bleue (accélération)/Code")
sys.path.append("./SIMS/Partie Verte (déviation magnétique)/Code")

try:
    import deviation2 # type: ignore
    import partie_electroaimant2 # type: ignore
except ImportError as e:
    print(f"Erreur d'importation: {e}")
    print("Assurez-vous que les chemins sys.path sont corrects et que les fichiers existent.")
    sys.exit(1)


class ParticleApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simulateur SIMS - Déviations")
        self.root.geometry("1400x800") # Taille initiale

        # Style
        style = ttk.Style()
        style.theme_use('clam') # Ou 'alt', 'default', 'classic'
        style.configure("TButton", padding=6, relief="flat", background="#ccc")
        style.configure("TLabelframe.Label", font=('Helvetica', 12, 'bold'))
        style.configure("TLabel", padding=2)
        style.configure("Treeview.Heading", font=('Helvetica', 10, 'bold'))

        # Liste pour stocker les données des particules (mass_u: float, charge_e: float)
        self.particles_data = []

        # --- Structure principale ---
        main_paned_window = ttk.PanedWindow(root, orient=tk.HORIZONTAL)
        main_paned_window.pack(fill=tk.BOTH, expand=True)

        # --- Panneau de contrôle (gauche) ---
        control_panel = ttk.Frame(main_paned_window, width=350)
        main_paned_window.add(control_panel, weight=1)

        # --- Section Particules ---
        particle_frame = ttk.LabelFrame(control_panel, text="Gestion des Particules")
        particle_frame.pack(pady=10, padx=10, fill=tk.X)
        self.create_particle_widgets(particle_frame)

        # --- Section Onglets Simulations ---
        self.notebook = ttk.Notebook(control_panel)
        self.notebook.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        self.mag_tab = ttk.Frame(self.notebook)
        self.elec_tab = ttk.Frame(self.notebook)

        self.notebook.add(self.mag_tab, text='Déviation Magnétique')
        self.notebook.add(self.elec_tab, text='Déviation Électrique')

        self.create_magnetic_widgets(self.mag_tab)
        self.create_electric_widgets(self.elec_tab)

        # --- Panneau de Plot (droite) ---
        plot_panel = ttk.Frame(main_paned_window)
        main_paned_window.add(plot_panel, weight=3) # Donne plus de place au plot

        # --- Zone Matplotlib ---
        self.fig, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.fig, master=plot_panel)
        self.canvas_widget = self.canvas.get_tk_widget()
        self.canvas_widget.pack(fill=tk.BOTH, expand=True)

        # --- Barre d'outils Matplotlib ---
        toolbar = NavigationToolbar2Tk(self.canvas, plot_panel)
        toolbar.update()
        toolbar.pack(side=tk.BOTTOM, fill=tk.X)

        # --- Barre de Statut ---
        self.status_var = tk.StringVar()
        self.status_var.set("Prêt.")
        status_bar = ttk.Label(root, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)

    # --- Widgets pour la gestion des particules ---
    def create_particle_widgets(self, parent):
        input_frame = ttk.Frame(parent)
        input_frame.pack(pady=5, padx=5, fill=tk.X)

        ttk.Label(input_frame, text="Masse (u):").grid(row=0, column=0, padx=5, sticky=tk.W)
        self.mass_entry = ttk.Entry(input_frame, width=10)
        self.mass_entry.grid(row=0, column=1, padx=5)
        self.mass_entry.insert(0, "28.0") # Exemple Si+

        ttk.Label(input_frame, text="Charge (e):").grid(row=0, column=2, padx=5, sticky=tk.W)
        self.charge_entry = ttk.Entry(input_frame, width=10)
        self.charge_entry.grid(row=0, column=3, padx=5)
        self.charge_entry.insert(0, "1.0")

        add_btn = ttk.Button(input_frame, text="Ajouter", command=self.add_particle)
        add_btn.grid(row=0, column=4, padx=10)

        # Treeview pour afficher les particules
        tree_frame = ttk.Frame(parent)
        tree_frame.pack(pady=5, padx=5, fill=tk.BOTH, expand=True)

        self.particle_tree = ttk.Treeview(tree_frame, columns=('Mass (u)', 'Charge (e)'), show='headings', height=5)
        self.particle_tree.heading('Mass (u)', text='Masse (u)')
        self.particle_tree.heading('Charge (e)', text='Charge (e)')
        self.particle_tree.column('Mass (u)', width=80, anchor=tk.CENTER)
        self.particle_tree.column('Charge (e)', width=80, anchor=tk.CENTER)

        # Scrollbar pour Treeview
        scrollbar = ttk.Scrollbar(tree_frame, orient=tk.VERTICAL, command=self.particle_tree.yview)
        self.particle_tree.configure(yscrollcommand=scrollbar.set)

        self.particle_tree.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        remove_btn = ttk.Button(parent, text="Supprimer sélection", command=self.remove_particle)
        remove_btn.pack(pady=5)

    # --- Widgets pour la déviation magnétique ---
    def create_magnetic_widgets(self, parent):
        frame = ttk.Frame(parent, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)

        # Vitesse initiale (Mise à jour valeur défaut)
        self.v0_mag_var = tk.StringVar(value="1.85e5") # Pour Si+ @ 5keV
        self.add_labeled_entry(frame, "Vitesse Initiale (m/s):", self.v0_mag_var).pack(fill=tk.X, pady=3)

        # Champ Magnétique (Slider) (Mise à jour plage et valeur défaut)
        ttk.Label(frame, text="Champ Magnétique Bz (T):").pack(anchor=tk.W, pady=(5,0))
        slider_frame_bz = ttk.Frame(frame)
        slider_frame_bz.pack(fill=tk.X, pady=(0,5))
        self.bz_var = tk.DoubleVar(value=0.21) # Pour Si+ @ 5keV, R=0.25m
        self.bz_slider = ttk.Scale(slider_frame_bz, from_=0.01, to=0.5, orient=tk.HORIZONTAL, variable=self.bz_var, command=self._on_bz_slider_change) # Plage 0.01-0.5T
        self.bz_slider.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 10))
        self.bz_label_var = tk.StringVar(value=f"{self.bz_var.get():.3f} T")
        ttk.Label(slider_frame_bz, textvariable=self.bz_label_var, width=10).pack(side=tk.LEFT)

        # --- AJOUT DU CHAMP X DETECTEUR ---
        self.x_detecteur_var = tk.StringVar(value="0.25") # Défaut réaliste (R=25cm)
        self.add_labeled_entry(frame, "X Détecteur (m):", self.x_detecteur_var).pack(fill=tk.X, pady=3)
        # --- FIN AJOUT ---

        # Domaine x (gardés pour info, mais non utilisés dans l'appel actuel)
        self.xmin_mag_var = tk.StringVar(value="0.0")
        # self.add_labeled_entry(frame, "X min (m):", self.xmin_mag_var).pack(fill=tk.X, pady=3) # Commenté pour alléger
        self.xmax_mag_var = tk.StringVar(value="0.5") # Ce champ n'est plus utilisé directement
        # self.add_labeled_entry(frame, "X max (m):", self.xmax_mag_var).pack(fill=tk.X, pady=3) # Commenté pour alléger

        # Bouton Tracer
        trace_btn = ttk.Button(frame, text="Tracer Déviation Magnétique", command=self.run_magnetic_simulation)
        trace_btn.pack(pady=15)

    # --- CALLBACK pour slider Bz (inchangé) ---
    def _on_bz_slider_change(self, event=None):
        self._update_bz_label()
        if self.particles_data:
            self.run_magnetic_simulation(called_by_slider=True)

    def _update_bz_label(self, event=None):
        self.bz_label_var.set(f"{self.bz_var.get():.3f} T")

    # --- Widgets pour la déviation électrique ---
    def create_electric_widgets(self, parent):
        # Utilisons le scénario de détection comme exemple réaliste
        frame = ttk.Frame(parent, padding="10")
        frame.pack(fill=tk.BOTH, expand=True)

        # Vitesse initiale
        self.v0_elec_var = tk.StringVar(value="1.85e5") # Pour Si+ @ 5keV
        self.add_labeled_entry(frame, "Vitesse Initiale (m/s):", self.v0_elec_var).pack(fill=tk.X, pady=3)

        # Angle initial
        self.angle_var = tk.StringVar(value="0.0") # Idéalement 0 pour détection
        self.add_labeled_entry(frame, "Angle Initial (° vs y):", self.angle_var).pack(fill=tk.X, pady=3)

        # Hauteur initiale
        self.y0_var = tk.StringVar(value="0.01") # Distance fente-détecteur
        self.add_labeled_entry(frame, "Hauteur Initiale (m):", self.y0_var).pack(fill=tk.X, pady=3)

        # Potentiel (Slider)
        ttk.Label(frame, text="Potentiel Détecteur (V):").pack(anchor=tk.W, pady=(5,0)) # Renommé pour clarté
        slider_frame_v = ttk.Frame(frame)
        slider_frame_v.pack(fill=tk.X, pady=(0,5))
        self.pot_var = tk.DoubleVar(value=-2000) # Tension typique MCP
        self.pot_slider = ttk.Scale(slider_frame_v, from_=-2500, to=0, orient=tk.HORIZONTAL, variable=self.pot_var, command=self._on_pot_slider_change) # Plage ajustée
        self.pot_slider.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 10))
        self.pot_label_var = tk.StringVar(value=f"{self.pot_var.get():.0f} V")
        ttk.Label(slider_frame_v, textvariable=self.pot_label_var, width=10).pack(side=tk.LEFT)

        # Distance (correspond à y0 si l'ion part de y=y0 et le détecteur est à y=0)
        self.dist_var = tk.StringVar(value="0.01")
        self.add_labeled_entry(frame, "Distance au Détecteur (m):", self.dist_var).pack(fill=tk.X, pady=3) # Renommé

        # Bouton Tracer
        trace_btn = ttk.Button(frame, text="Tracer Déviation Électrique", command=self.run_electric_simulation)
        trace_btn.pack(pady=15)

    # --- CALLBACK pour slider Potentiel (inchangé) ---
    def _on_pot_slider_change(self, event=None):
        self._update_pot_label()
        if self.particles_data:
            self.run_electric_simulation(called_by_slider=True)

    def _update_pot_label(self, event=None):
        self.pot_label_var.set(f"{self.pot_var.get():.0f} V")

    # --- Helper pour ajouter Label + Entry (inchangé) ---
    def add_labeled_entry(self, parent, label_text, string_var):
        entry_frame = ttk.Frame(parent)
        ttk.Label(entry_frame, text=label_text, width=20).pack(side=tk.LEFT, padx=5)
        entry = ttk.Entry(entry_frame, textvariable=string_var)
        entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        return entry_frame

    # --- Logique métier ---
    def add_particle(self):
        try:
            mass_u = float(self.mass_entry.get())
            charge_e = float(self.charge_entry.get())

            if mass_u <= 0:
                raise ValueError("Masse doit être > 0.")
            # Permettre charge négative si nécessaire pour certains SIMS (ions primaires négatifs)
            # Mais pour l'analyse secondaire, c'est souvent positif.
            if charge_e == 0:
                 raise ValueError("Charge doit être != 0.")

            particle_info = (mass_u, charge_e)
            if particle_info not in self.particles_data :
                self.particles_data.append(particle_info)
                self.particle_tree.insert('', tk.END, values=(f"{mass_u:.3f}", f"{charge_e:+.2f}"))
                self.status_var.set(f"Particule ajoutée: {mass_u:.3f} u, {charge_e:+.2f} e")
            else :
                messagebox.showwarning("Doublon", "Cette particule est déjà dans la liste.")
                self.status_var.set("Ajout annulé (doublon).")

        except ValueError as e:
            messagebox.showerror("Erreur d'entrée", f"Entrée invalide : {e}")
            self.status_var.set("Erreur d'ajout de particule.")

    def remove_particle(self):
        selected_items = self.particle_tree.selection()
        if not selected_items:
            messagebox.showwarning("Aucune sélection", "Veuillez sélectionner une particule à supprimer.")
            return

        indices_to_remove = []
        items_to_remove_tree = []

        for item_id in selected_items:
            values = self.particle_tree.item(item_id, 'values')
            try:
                mass_u_str, charge_e_str = values
                mass_u = float(mass_u_str)
                charge_e = float(charge_e_str)
                particle_tuple = (mass_u, charge_e)
                # Cherche l'index en comparant les tuples (après conversion)
                found = False
                for i, data_tuple in enumerate(self.particles_data):
                    if abs(data_tuple[0] - mass_u) < 1e-6 and abs(data_tuple[1] - charge_e) < 1e-6:
                        indices_to_remove.append(i)
                        items_to_remove_tree.append(item_id)
                        found = True
                        break # Trouvé, passe à l'item suivant
                if not found:
                    print(f"Avertissement: Particule {particle_tuple} sélectionnée mais non trouvée dans les données (problème de précision?).")
            except (ValueError, IndexError, TypeError):
                 print(f"Erreur lors de la récupération des données pour l'item {item_id}")

        # Supprimer de la liste de données (en partant de la fin)
        indices_to_remove.sort(reverse=True)
        removed_count = 0
        for index in indices_to_remove:
            if 0 <= index < len(self.particles_data):
                del self.particles_data[index]
                removed_count += 1
            else:
                print(f"Avertissement: Index {index} hors limites lors de la suppression.")

        # Supprimer du Treeview
        for item_id in items_to_remove_tree:
            try:
                self.particle_tree.delete(item_id)
            except tk.TclError:
                print(f"Avertissement: Impossible de supprimer l'item {item_id} du Treeview (déjà supprimé?).")


        self.status_var.set(f"{removed_count} particule(s) supprimée(s).")


    # --- MODIFICATION : Utilisation de x_detecteur_var ---
    def run_magnetic_simulation(self, called_by_slider=False):
        if not self.particles_data:
            if not called_by_slider:
                messagebox.showwarning("Aucune particule", "Veuillez ajouter au moins une particule.")
            self.status_var.set("Ajoutez des particules pour simuler.")
            self.ax.cla()
            self.canvas.draw()
            return

        try:
            v0 = float(self.v0_mag_var.get())
            bz = self.bz_var.get()
            # --- Lecture de la nouvelle variable ---
            x_detecteur = float(self.x_detecteur_var.get())

            if v0 <= 0 : raise ValueError("V0 > 0 requis.")
            # --- Validation de la nouvelle variable ---
            if x_detecteur <= 0 : raise ValueError("X Détecteur > 0 requis.")

            particles_ue = [(p[0], p[1]) for p in self.particles_data]

            self.ax.cla()
            self.status_var.set("Calcul déviation magnétique en cours...")
            self.root.update_idletasks()

            # --- Utilisation de x_detecteur dans l'appel backend ---
            partie_electroaimant2.tracer_ensemble_trajectoires(
                particles_ue, v0, bz, x_detecteur, ax=self.ax, create_plot=False
            )
            # --- FIN MODIFICATION APPEL ---

            self.canvas.draw()
            self.status_var.set("Tracé déviation magnétique terminé.")

        except ValueError as e:
            if not called_by_slider:
                messagebox.showerror("Erreur de paramètre", f"Paramètre invalide : {e}")
            self.status_var.set(f"Erreur paramètre (Mag): {e}")
        except Exception as e:
            if not called_by_slider:
                messagebox.showerror("Erreur de Simulation", f"Une erreur est survenue (Mag): {e}")
            print(f"Erreur Simulation Magnétique: {type(e).__name__}: {e}")
            self.status_var.set("Erreur de simulation magnétique.")


    # --- Simulation Électrique (inchangée dans sa logique principale) ---
    def run_electric_simulation(self, called_by_slider=False):
        if not self.particles_data:
            if not called_by_slider:
                messagebox.showwarning("Aucune particule", "Veuillez ajouter au moins une particule.")
            self.status_var.set("Ajoutez des particules pour simuler.")
            self.ax.cla()
            self.canvas.draw()
            return

        try:
            v0 = float(self.v0_elec_var.get())
            angle_deg = float(self.angle_var.get()) # Gardons la convention de l'UI (vs +y)
            y0 = float(self.y0_var.get())
            potentiel = self.pot_var.get()
            distance = float(self.dist_var.get())

            if v0 <= 0 : raise ValueError("V0 > 0 requis.")
            if y0 <= 0 : raise ValueError("Hauteur Initiale > 0 requis.")
            if distance <= 0 : raise ValueError("Distance > 0 requis.")
            # La validation de l'angle dépend de ce que deviation2 attend réellement.
            # Si angle vs +y, 0 <= angle < 90 est typique.
            if not (0 <= angle_deg < 90):
                 raise ValueError("Angle doit être entre 0° (inclus) et 90° (exclus).")

            angle_rad = np.radians(angle_deg)

            E = deviation2.champ_electrique_v2(distance, potentiel)

            particles_ue = [(p[0], p[1]) for p in self.particles_data]

            self.ax.cla()
            self.status_var.set("Calcul déviation électrique en cours...")
            self.root.update_idletasks()

            deviation2.tracer_ensemble_trajectoires(
                particles_ue, v0, angle_rad, y0, E, ax=self.ax
            )

            self.canvas.draw()
            self.status_var.set("Tracé déviation électrique terminé.")

        except ValueError as e:
            if not called_by_slider:
                messagebox.showerror("Erreur de paramètre", f"Paramètre invalide : {e}")
            self.status_var.set(f"Erreur paramètre (Elec): {e}")
        except Exception as e:
            if not called_by_slider:
                messagebox.showerror("Erreur de Simulation", f"Une erreur est survenue (Elec): {e}")
            print(f"Erreur Simulation Électrique: {type(e).__name__}: {e}")
            self.status_var.set("Erreur de simulation électrique.")


if __name__ == "__main__":
    root = tk.Tk()
    app = ParticleApp(root)
    root.mainloop()